## StarTracker WCS solutions - 31-Jan-23

Craig Lage - Reworked to work with the new summit_utils code

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
from scipy.ndimage import median_filter
import matplotlib.pyplot as plt
import astropy.io.fits as pf
import pandas as pd
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION

import lsst.afw.image as afwImage
from lsst.geom import SpherePoint
from lsst.geom import Angle as afwAngle
from astroquery.simbad import Simbad

import lsst.afw.image as afwImage
from lsst.summit.utils.utils import starTrackerFileToExposure
from lsst.summit.utils.astrometry.anet import CommandLineSolver
from lsst.summit.utils.astrometry.utils import runCharactierizeImage, filterSourceCatOnBrightest
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig

In [ ]:
solver = CommandLineSolver('/project/shared/ref_cats/astrometry_net/')

In [ ]:
#cameras = [['Wide', 101], ['Narrow', 102]]
cameras = [['Narrow', 102]]
dates = [20230321]
for [camera, num] in cameras:
    for date in dates:
        year = int(date/10000)
        month = int((date - 10000 * year)/100)
        day = int((date - 10000 * year - 100 * month))
        path = f"/project/GenericCamera/{num}/{year}/{month:02}/{day:02}/"
        for seqNum in range(1160, 1161):
            filename = path + f"GC{num}_O_{date}_{seqNum:06}.fits"
            narrow_exp = starTrackerFileToExposure(filename)
            results = runCharactierizeImage(narrow_exp, 5, 25)
            filteredCat = filterSourceCatOnBrightest(results.sourceCat, 0.2, maxSources=100)
            # Need to loosen percentageScaleError because header SECPIX is wrong
            solverResults = solver.run(narrow_exp, filteredCat, percentageScaleError=60.0, isWideField=True)


In [ ]:
narrow_wcs = solverResults.wcs

In [ ]:
def runCharactierizeImage(exp, snr, minPix):
    """Run the image characterization task, finding only bright sources.

    Parameters
    ----------
    exp : `lsst.afw.image.Exposure`
        The exposure to characterize.
    snr : `float`
        The SNR threshold for detection.
    minPix : `int`
        The minimum number of pixels to count as a source.

    Returns
    -------
    result : `lsst.pipe.base.Struct`
        The result from the image characterization task.
    """
    charConfig = CharacterizeImageConfig()
    charConfig.doMeasurePsf = False
    charConfig.doApCorr = False
    charConfig.doDeblend = False
    charConfig.repair.doCosmicRay = False

    charConfig.detection.minPixels = minPix
    charConfig.detection.thresholdValue = snr
    charConfig.detection.includeThresholdMultiplier = 1
    charConfig.detection.nSigmaToGrow = 0

    charConfig.psfIterations = 1
    charConfig.installSimplePsf.fwhm = 5
    charConfig.installSimplePsf.width = 51
    
    # fit background with the most simple thing possible as we don't need
    # much sophistication here. weighting=False is *required* for very
    # large binSizes.
#     charConfig.background.algorithm = 'AKIMA'
    charConfig.background.approxOrderX = 1
    charConfig.background.approxOrderY = -1
    charConfig.background.binSize = max(exp.getWidth(), exp.getHeight())
    charConfig.background.weighting = False

    # set this to use all the same minimal settings as those above
    charConfig.detection.background = charConfig.background

    charTask = CharacterizeImageTask(config=charConfig)

    charResult = charTask.run(exp)
    return charResult

def solveFastCamera(filename, doPlot=False):
    exp = starTrackerFileToExposure(filename)
    exp.image.array = median_filter(exp.image.array, 2, mode='reflect')
    result = runCharactierizeImage(exp, 5, 25)
    sources = filterSourceCatOnBrightest(result.sourceCat, .5, minSources=5)
    if doPlot:
        plot(exp, sources, doSmooth=False)
    result = solver.run(exp, sources, True, useGaia=True, percentageScaleError=10, radius=5)
    if result is not None:
        fastWcs = result.wcs
        fastVisitInfo = exp.visitInfo
    else:
        print('fit failed')
        return None
    return fastWcs

# Starting again

In [ ]:
#Enter which image you want to look at
[camera, num] = ['Fast', 103]
#[camera, num] = ['Narrow', 102]
#[camera, num] = ['Wide', 101]
date = 20230321
seqNum = 1160


year = int(date/10000)
month = int((date - 10000 * year)/100)
day = int((date - 10000 * year - 100 * month))

path = f"/project/GenericCamera/{num}/{year}/{month:02}/{day:02}/"
print(path)
filename = path + f"GC{num}_O_{date}_{seqNum:06}.fits"
fast_exp = starTrackerFileToExposure(filename)
result = solveFastCamera(filename, doPlot=False)


## Now calculate the deltas from the narrow camera to the fast camera.

In [ ]:
# This is a back-up
#from lsst.afw.geom import SkyWcs
#fast_wcs = SkyWcs.readFits('/home/mfisherlevine/temp/fastCamFits/GC103_O_20230321_001160_wcs.fits')

In [ ]:
fast_wcs = result

In [ ]:
rot = fast_wcs.getRelativeRotationToWcs(narrow_wcs)

In [ ]:
rot.asRadians()

In [ ]:
fast_sky_origin = fast_wcs.getSkyOrigin()

In [ ]:
fast_sky_origin

In [ ]:
fast_wcs.getPixelScale().asArcseconds() / narrow_wcs.getPixelScale().asArcseconds()

In [ ]:
test.asArcseconds()

In [ ]:
narrow_wcs.skyToPixel(fast_sky_origin)


In [ ]:
fast_exp.image.array.shape

In [ ]:
def fast_corners(fast_wcs, narrow_wcs):
    scale_ratio = fast_wcs.getPixelScale().asArcseconds() / narrow_wcs.getPixelScale().asArcseconds()
    theta = fast_wcs.getRelativeRotationToWcs(narrow_wcs).asRadians()
    center = narrow_wcs.skyToPixel(fast_wcs.getSkyOrigin())
    dx0 = fast_wcs.getPixelOrigin().x * scale_ratio
    dy0 = fast_wcs.getPixelOrigin().y * scale_ratio
    d0 = np.array([dx0, dy0])
    rot = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])

    d = np.dot(rot, d0)
    dx = d[0]; dy = d[1]
    print(dx0, dy0)
    print(dx, dy)
    LL = [center.x-dx, center.y-dy]
    UL = [center.x-dx, center.y+dy]
    UR = [center.x+dx, center.y+dy]
    LR = [center.x+dx, center.y-dy]
    return [LL,UL,UR,LR,LL]

In [ ]:
# Now plot the data with matplotlib
%matplotlib inline
from matplotlib.path import Path
from matplotlib.patches import PathPatch
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

fig, ax = plt.subplots(figsize=(16,16))
ax.set_title(f"StarTracker {camera}, SeqNum {seqNum}")
ax.imshow(narrow_exp.image.array,  interpolation='Nearest', cmap='gray', vmin=0.1, vmax=1, origin='lower')
path = Path(fast_corners(fast_wcs, narrow_wcs))
patch = PathPatch(path, facecolor='none', edgecolor='y')
ax.add_patch(patch)
im =plt.imshow(fast_exp.image.array,  interpolation='Nearest', cmap='gray', vmin=1, vmax=1000, origin='lower', \
               extent=[0,narrow_exp.image.array.shape[1],0,narrow_exp.image.array.shape[0]], clip_path=patch, clip_on=True, alpha=0.5)
im.set_clip_path(patch)


#im = ax.imshow(image)
#patch = patches.Circle((260, 200), radius=200, transform=ax.transData)
#im.set_clip_path(patch)

#plt.imshow(fast_exp.image.array, alpha=0.5, interpolation='Nearest', cmap='gray', vmin=1, vmax=1000)
#plt.scatter(xs, ys\
#            ,facecolors='none', edgecolors='g', s=1000, lw=2)
#plt.scatter(boresight[0], boresight[1]\
#            ,facecolors='r', edgecolors='r', marker='X', s=100, lw=2)
#plt.scatter(shiftX, shiftY\
#            ,facecolors='y', edgecolors='y', marker='X', s=100, lw=2)

#for x, y, name in zip(xs, ys, names):
#    plt.text(x+50,y, name, color='g', fontsize = 36)
#colorbar(ax)
plt.tight_layout(h_pad=1)
#plt.savefig(f"/home/craiglagegit/DATA/StarTracker_Labels_Simbad_{camera}_{date}_{seqNum}.pdf")